In [1]:
import pandas as pd
import ast
import pymorphy3
import nltk
from nltk.corpus import stopwords
import re
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score

c:\Users\moder\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_parquet('tengri_data.parquet')
df.head()

,title,link,content,tags,date
0,Алабай перепрыгнул во двор соседей и загрыз дв...,https://tengrinews.kz/kazakhstan_news/alabay-p...,В Талдыкоргане сорвавшийся с цепи алабай прони...,"[нападение, Собаки, Животные, Талдыкорган, Гиб...",2026-01-28T21:07:51+05:00
1,Air Astana предупредила о задержках рейсов из ...,https://tengrinews.kz/kazakhstan_news/air-asta...,Казахстанская авиакомпания Air Astana предупре...,"[Алматы, Air Astana, Задержка, Погода]",2026-01-28T20:40:02+05:00
2,"Перегрев экономики, изменения в Конституции и ...",https://tengrinews.kz/kazakhstan_news/peregrev...,Tengrinews.kz подготовил дайджест актуальных н...,"[Касым-Жомарт Токаев, Алматы, Шымкент, Футбол,...",2026-01-28T20:15:20+05:00
3,“Дело Нурай“: на подозреваемого завели еще одн...,https://tengrinews.kz/kazakhstan_news/delo-nur...,На подозреваемого в убийстве 21-летней Нурай С...,"[Убийство, Сталкинг, Похищение]",2026-01-28T19:29:00+05:00
4,“Отель Кайрата Сатыбалды“ в Алматы: зданию наш...,https://tengrinews.kz/kazakhstan_news/otel-kay...,"Стало известно, что в отеле на берегу реки Есе...","[Кайрат Сатыбалдыулы, Отель, Алматы, Суд, Акимат]",2026-01-28T18:50:22+05:00


In [3]:
df['tags'] = df['tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
for i in df.tags.head():
  for j in i:
    print(j)

нападение
Собаки
Животные
Талдыкорган
Гибель
Полиция
Жалоба
Соцсети
Алматы
Air Astana
Задержка
Погода
Касым-Жомарт Токаев
Алматы
Шымкент
Футбол
Спорт
Уголовное дело
Убийство
Кайрат Сатыбалдыулы
Отель
реформа
Конституция
АФМ
Экономика
Убийство
Сталкинг
Похищение
Кайрат Сатыбалдыулы
Отель
Алматы
Суд
Акимат


In [4]:
tags_count = dict()
for tag_list in df.tags:
    for tag in tag_list:
        if tag in tags_count:
            tags_count[tag] += 1
        else:
            tags_count[tag] = 1
print(sum(tags_count.values()))

357214


In [5]:
df.isnull().sum()

title        0
link         0
content      0
tags         0
date       169
dtype: int64

In [6]:
df = df.dropna(subset=['content', 'tags'])
df = df[df['content'].str.strip() != ""]
df = df[df['tags'].map(len) > 0]
initial_count = len(df)
df = df.drop_duplicates(subset=['content'])
print(f"Удалено полных дубликатов: {initial_count - len(df)}")

Удалено полных дубликатов: 5


In [7]:
from difflib import SequenceMatcher

def are_similar(string1, string2, threshold=0.95):
    return SequenceMatcher(None, string1, string2).quick_ratio() > threshold

df['content_len'] = df['content'].str.len()
df = df.sort_values(by='content_len').reset_index(drop=True)

to_drop = []

contents = df['content'].tolist()
indices = df.index.tolist()

for i in range(len(contents) - 1):
    if are_similar(contents[i], contents[i+1]):
        to_drop.append(indices[i])

df = df.drop(to_drop)
df = df.drop(columns=['content_len'])
print(f"Удалено нечетких дубликатов: {len(to_drop)}")

Удалено нечетких дубликатов: 212


In [8]:
df.head()

,title,link,content,tags,date
0,Информация,https://tengrinews.kz/kazakhstan_news/informat...,"Информация удалена, так как первоисточник - ве...","[Алматы, Полиция, Мошенничество, Розыск]",2022-09-15T23:52:04+05:00
1,Очереди образовались на АЗС Алматы,https://tengrinews.kz/kazakhstan_news/ocheredi...,На автозаправочных станциях в Алматы наблюдают...,"[АЗС, заправки, очереди, Алматы]",2024-01-23T00:52:12+05:00
2,О регистрации автомобилей с иностранными номер...,https://tengrinews.kz/kazakhstan_news/registra...,На площадке СЦК проводится пресс-конференция п...,"[номера, Автомобили, Регистрация]",2020-08-21T14:27:09+05:00
3,Совместное заседание палат Парламента проходит...,https://tengrinews.kz/kazakhstan_news/sovmestn...,Председатель Мажилиса Ерлан Кошанов открыл сов...,"[Парламент, заседание]",2023-06-20T15:08:25+05:00
4,Суточный прирост заражений коронавирусом снизи...,https://tengrinews.kz/kazakhstan_news/sutochny...,В Казахстане за прошедшие сутки зарегистрирова...,"[Коронавирус, Статистика, Минздрав]",2022-02-27T08:04:59+05:00


In [9]:
morph = pymorphy3.MorphAnalyzer()

def lemmatize_tags(tags):
    if isinstance(tags, str):
        try:
            tags = ast.literal_eval(tags)
        except:
            return []

    clean_results = []
    
    for tag in tags:
        words = tag.split()
        is_person = False
        
        for w in words:
            parsed = morph.parse(w)[0]
            if any(grammem in parsed.tag for grammem in ['Name', 'Surn', 'Patr']):
                is_person = True
                break
        
        if not is_person:
            lemmatized_tag = " ".join([morph.parse(w)[0].normal_form for w in words])
            clean_results.append(lemmatized_tag.capitalize())
        else:

            clean_results.append(tag.capitalize())

    return list(set(clean_results))

df['clean_tags'] = df['tags'].apply(lemmatize_tags)
df = df[df['clean_tags'].map(len) > 0]

In [10]:
for i in df['clean_tags'].head():
  for j in i:
    print(j)

Полиция
Мошенничество
Алматы
Розыск
Заправка
Алматы
Азс
Очередь
Номер
Регистрация
Автомобиль
Заседание
Парламент
Минздрав
Коронавирус
Статистика


In [11]:
df.shape

(86233, 6)

In [12]:
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

custom_stops = [
    'тенериньюс', 'tengrinews', 'кз', 'kz', 'фото', 'видео', 'читайте',
    'подписывайтесь', 'инстаграм', 'телеграм', 'новости', 'страница',
    'сообщать', 'передавать', 'рассказывать', 'заявлять', 'отмечать',
    'корреспондент', 'пресс-служба', 'подробности', 'ранее', 'пишет',
    'говорится', 'согласно', 'слово', 'мнение', 'пояснить', 'уточнять',
    'сегодня', 'вчера', 'завтра', 'понедельник', 'вторник', 'среда',
    'четверг', 'пятница', 'суббота', 'воскресенье', 'месяц', 'год'
]

stop_words.update(custom_stops)

def remove_stops(text_list):
    return [word for word in text_list if word not in stop_words]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\moder\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
def normalize_text(text):
    if not text:
        return ""

    text = re.sub(r'[^а-яА-ЯёЁ\s]', ' ', str(text).lower())

    words = text.split()

    normalized_words = []
    for word in words:
        if word not in stop_words and len(word) > 2:

            p = morph.parse(word)[0]
            normalized_words.append(p.normal_form)

    return " ".join(normalized_words)

In [14]:
tqdm.pandas()

print("Начинаю нормализацию текста...")
df['clean_content'] = df['content'].progress_apply(normalize_text)

print(df[['content', 'clean_content']].head(3))

Начинаю нормализацию текста...


100%|██████████| 86233/86233 [29:47<00:00, 48.24it/s] 

                                             content  \
0  Информация удалена, так как первоисточник - ве...   
1  На автозаправочных станциях в Алматы наблюдают...   
2  На площадке СЦК проводится пресс-конференция п...   

                                       clean_content  
0  информация удалить первоисточник ведомственный...  
1  автозаправочный станция алматы наблюдаться оче...  
2  площадка сцк проводиться пресс конференция тем...  


In [15]:
df.to_parquet('tengri_data_cleaned.parquet')

In [16]:
# df = pd.read_parquet('tengri_data_cleaned.parquet')
#df['tags'] = df['tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
#df['clean_tags'] = df['clean_tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df.head()

,title,link,content,tags,date,clean_tags,clean_content
0,Информация,https://tengrinews.kz/kazakhstan_news/informat...,"Информация удалена, так как первоисточник - ве...","[Алматы, Полиция, Мошенничество, Розыск]",2022-09-15T23:52:04+05:00,"[Полиция, Мошенничество, Алматы, Розыск]",информация удалить первоисточник ведомственный...
1,Очереди образовались на АЗС Алматы,https://tengrinews.kz/kazakhstan_news/ocheredi...,На автозаправочных станциях в Алматы наблюдают...,"[АЗС, заправки, очереди, Алматы]",2024-01-23T00:52:12+05:00,"[Заправка, Алматы, Азс, Очередь]",автозаправочный станция алматы наблюдаться оче...
2,О регистрации автомобилей с иностранными номер...,https://tengrinews.kz/kazakhstan_news/registra...,На площадке СЦК проводится пресс-конференция п...,"[номера, Автомобили, Регистрация]",2020-08-21T14:27:09+05:00,"[Номер, Регистрация, Автомобиль]",площадка сцк проводиться пресс конференция тем...
3,Совместное заседание палат Парламента проходит...,https://tengrinews.kz/kazakhstan_news/sovmestn...,Председатель Мажилиса Ерлан Кошанов открыл сов...,"[Парламент, заседание]",2023-06-20T15:08:25+05:00,"[Заседание, Парламент]",председатель мажилис ерлан кошановый открыть с...
4,Суточный прирост заражений коронавирусом снизи...,https://tengrinews.kz/kazakhstan_news/sutochny...,В Казахстане за прошедшие сутки зарегистрирова...,"[Коронавирус, Статистика, Минздрав]",2022-02-27T08:04:59+05:00,"[Минздрав, Коронавирус, Статистика]",казахстан прошедшее сутки зарегистрировать нов...


In [1]:
from nnTag import FastTagPredictor

c:\Users\moder\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
predictor = FastTagPredictor()
predictor.train(df, epochs=30, batch_size=16)

Начинаем


Loading weights: 100%|██████████| 199/199 [00:00<00:00, 563.41it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Устройство: cuda
Подготовка данных...
Создание эмбеддингов...


Batches: 100%|██████████| 2695/2695 [09:16<00:00,  4.84it/s]


Подготовка меток...
Начинаем обучение...
Эпоха 1/30 - Loss: 0.0088, Val Loss: 0.0047, Val Acc: 0.9980
Эпоха 2/30 - Loss: 0.0044, Val Loss: 0.0038, Val Acc: 0.9977
Эпоха 3/30 - Loss: 0.0038, Val Loss: 0.0036, Val Acc: 0.9979
Эпоха 4/30 - Loss: 0.0036, Val Loss: 0.0034, Val Acc: 0.9979
Эпоха 5/30 - Loss: 0.0034, Val Loss: 0.0033, Val Acc: 0.9977
Эпоха 6/30 - Loss: 0.0033, Val Loss: 0.0034, Val Acc: 0.9981
Эпоха 7/30 - Loss: 0.0033, Val Loss: 0.0033, Val Acc: 0.9978
Эпоха 8/30 - Loss: 0.0032, Val Loss: 0.0033, Val Acc: 0.9979
Эпоха 9/30 - Loss: 0.0032, Val Loss: 0.0033, Val Acc: 0.9979
Эпоха 10/30 - Loss: 0.0032, Val Loss: 0.0033, Val Acc: 0.9980
Эпоха 11/30 - Loss: 0.0031, Val Loss: 0.0033, Val Acc: 0.9980
Эпоха 12/30 - Loss: 0.0031, Val Loss: 0.0033, Val Acc: 0.9979
Эпоха 13/30 - Loss: 0.0031, Val Loss: 0.0033, Val Acc: 0.9978
Эпоха 14/30 - Loss: 0.0031, Val Loss: 0.0033, Val Acc: 0.9980
Эпоха 15/30 - Loss: 0.0031, Val Loss: 0.0033, Val Acc: 0.9979
Эпоха 16/30 - Loss: 0.0030, Val Loss: 

In [21]:
predictor.save()

Модель сохранена: pytorch_tag_model.pth


In [2]:
mmodel = FastTagPredictor()

mmodel.load('pytorch_tag_model')


Начинаем


Loading weights: 100%|██████████| 199/199 [00:00<00:00, 733.88it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Устройство: cuda
Модель загружена: pytorch_tag_model.pth


In [7]:
text = """
Глава государства уверен, что изменения в Конституцию выведут развитие Казахстана на качественно новый уровень, перезагрузят государственную систему и позволят выстроить более совершенное взаимодействие власти и общества.

Особое внимание в проекте конституционных поправок уделено понятию ответственного и созидательного патриотизма.
"Патриотизм – это не просто лозунги. Нужно относиться очень ответственно к этому святому понятию и созидать. Об этом я говорил на недавнем заседании Национального курултая. Ответственность и созидание – вот что отличает патриотичного гражданина", - сказал Токаев.

Президент отметил, что каждый волонтер является ярким примером истинного патриотизма: стремление делать добро и улучшать мир — ключевой показатель любви к Отечеству.

"Известнейший общественный деятель пастор Мартин Лютер Кинг говорил: "I have a dream" ("У меня есть мечта"). Так что мечтайте сделать этот мир лучше. Я полагаю, что вы добьетесь этого", - обратился Токаев к волонтерам.

"I have a dream" — это знаменитая фраза, произнесенная американским пастором и лидером движения за гражданские права Мартином Лютером Кингом 28 августа 1963 года во время марша на Вашингтон. В своей речи Кинг выразил мечту о мире, равенстве и справедливости, где люди всех рас и национальностей будут жить вместе без дискриминации. Фраза стала символом борьбы за гражданские права и вдохновляет людей по всему миру стремиться к социальной справедливости и личной ответственности."""
tags = mmodel.predict(text, threshold=0.05, boost_factor=1)
print(f"Текст: {text}")
print(f"Теги:")
for tag, score in tags:
    print(f"   • {tag}: {score:.3f}")

Текст: 
Глава государства уверен, что изменения в Конституцию выведут развитие Казахстана на качественно новый уровень, перезагрузят государственную систему и позволят выстроить более совершенное взаимодействие власти и общества.

Особое внимание в проекте конституционных поправок уделено понятию ответственного и созидательного патриотизма.
"Патриотизм – это не просто лозунги. Нужно относиться очень ответственно к этому святому понятию и созидать. Об этом я говорил на недавнем заседании Национального курултая. Ответственность и созидание – вот что отличает патриотичного гражданина", - сказал Токаев.

Президент отметил, что каждый волонтер является ярким примером истинного патриотизма: стремление делать добро и улучшать мир — ключевой показатель любви к Отечеству.

"Известнейший общественный деятель пастор Мартин Лютер Кинг говорил: "I have a dream" ("У меня есть мечта"). Так что мечтайте сделать этот мир лучше. Я полагаю, что вы добьетесь этого", - обратился Токаев к волонтерам.

"I ha